
Open this on Google Colab: https://colab.research.google.com/drive/1zgMUH2DQCquWvLgJc2NBup0dDSnofvN0?usp=sharing  
 
Model on HuggingFace: https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english 

## Libraries Explained

- **dotenv**: Loads environment variables from a `.env` file into the application's environment, helping manage configuration separately from code.

- **huggingface_hub**: 
  - **HfApi**: Provides programmatic access to the Hugging Face model hub for uploading, downloading, and managing models.
  - **hf_hub_download**: Simplifies downloading model files from the Hugging Face hub to your local environment.

- **transformers**: Offers pre-trained models for natural language processing tasks. The `pipeline` function specifically provides an easy-to-use interface for common NLP tasks like text generation, sentiment analysis, and question answering.


In [ ]:
import os, json, datetime
from datetime import datetime
from dotenv import load_dotenv

from huggingface_hub import HfApi
from huggingface_hub import hf_hub_download



from transformers import pipeline


# Loading Environment Variables for Hugging Face


This code snippet performs two essential operations:

1. `load_dotenv()` - Loads environment variables from a `.env` file into the application's environment. This is a common pattern for securely storing configuration and sensitive information outside of the source code.

2. `hf_key = os.getenv("HF_TOKEN")` - Retrieves the Hugging Face API token from the environment variables and assigns it to the variable `hf_key`. This token is required for authenticated access to the Hugging Face Hub services, including downloading private models or models with gated access.


In [ ]:
load_dotenv()
hf_key=os.getenv("HF_TOKEN")


# Hugging Face Model Reference

[distilbert/distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english)

This code defines a reference to a specific pre-trained model from the Hugging Face Model Hub. The model referenced is:

- **Model**: `distilbert-base-uncased-finetuned-sst-2-english`
- **Creator/Organization**: `distilbert`
- **Description**: This is a DistilBERT model that has been fine-tuned on the Stanford Sentiment Treebank v2 (SST-2) dataset for sentiment analysis tasks. It can classify text as either positive or negative sentiment.
- **Architecture**: DistilBERT (a distilled version of BERT that is smaller, faster, and requires fewer resources while maintaining good performance)
- **Use Case**: Sentiment analysis and classification


In [ ]:
hf_reference='distilbert/distilbert-base-uncased-finetuned-sst-2-english'


# Using Hugging Face Inference API


This code demonstrates how to use the Hugging Face Inference API for text classification:

1. **Import**: First, the `InferenceClient` class is imported from the `huggingface_hub` package.

2. **Client Initialization**: An `InferenceClient` object is created with:
   - `provider="hf-inference"` - Specifies using Hugging Face's hosted inference API
   - `api_key=hf_key` - Uses the previously loaded API token for authentication

3. **Text Classification**: The client performs sentiment analysis on the text "I like you. I love you" using the DistilBERT model defined earlier in `hf_reference`.

4. **Result**: The final line will display the classification results, which typically include the predicted labels (positive/negative) and their associated confidence scores.


In [ ]:
from huggingface_hub import InferenceClient
client = InferenceClient(provider="hf-inference",api_key=hf_key,)

result = client.text_classification(text="I like you. I love you",model=hf_reference)
result


# Downloading Specific Model Files from Hugging Face Hub


This code snippet demonstrates how to selectively download specific files from a Hugging Face model repository:

1. **File Definition**: First, a list of commonly required files for transformer models is defined, with comments explaining each file's purpose:
   - Vocabulary files for tokenization
   - Configuration files for model architecture
   - Tokenizer files for text preprocessing
   - Model weights in different formats (PyTorch and SafeTensors)

2. **Selective Download**: The code iterates through each file in the list and:
   - Attempts to download it using `hf_hub_download()`
   - Specifies the model repository via `repo_id=hf_reference`
   - Saves files to a local directory structure based on the model name
   - Prints the local path where each file is saved

3. **Error Handling**: The try-except block catches and reports any download failures, allowing the process to continue even if certain files aren't available for the specific model.


In [ ]:
# List of required files
required_files = [
    "vocab.txt",          # Vocabulary file (if applicable)
    "vocab.json",          # Vocabulary file (if applicable)       
    "config.json",        # Model configuration
    "tokenizer.json",     # Tokenizer configuration (if applicable)
    "tokenizer_config.json",     # Tokenizer configuration (if applicable)
    "merges.txt",         # BPE merge rules file (if applicable)
    "pytorch_model.bin",  # Model weights
    "model.safetensors",  # Alternative model weights format
]


# Download only the required files
for file_name in required_files:
    try:
        print()
        print(f"Attempting to download: {file_name}")
        local_path = hf_hub_download(repo_id=hf_reference, filename=file_name, local_dir=f"models/{hf_reference.split('/')[1]}")
        print(f"Saved to: {local_path}")
    except Exception as e:
        print(f"Could not download {file_name}: {e}")
        
    



# Creating Sentiment Analysis Pipelines


This code initializes two sentiment analysis pipelines using Hugging Face's `transformers` library:

1. **Cached Model Pipeline**: 
   - `hf_model_cache` uses the model identifier directly (`hf_reference`)
   - When this pipeline runs, it will first check the default Hugging Face cache directory on your system
   - If not found in cache, it automatically downloads the model from Hugging Face Hub

2. **Local Model Pipeline**:
   - `hf_model_local` uses the previously downloaded model files
   - Points to the local directory where model files were saved earlier
   - Loads the model from the local files rather than downloading or using cache
   - Path is constructed by extracting just the model name from the reference

Both pipelines provide the same sentiment analysis functionality but differ in where they source the model files from, allowing flexibility between network-dependent and offline usage.


In [ ]:
hf_model_cache = pipeline("sentiment-analysis", model=hf_reference)
hf_model_local = pipeline("sentiment-analysis", model=f"models/{hf_reference.split('/')[1]}")


# Testing the Sentiment Analysis Model


This code demonstrates how to use the local sentiment analysis pipeline:

1. **Input Text**: Defines a medical text about high blood sugar and blood pressure readings

2. **Model Inference**: Passes the text to the previously initialized local sentiment analysis model (`hf_model_local`)

3. **Output Display**: Prints the classification results, which typically include:
   - The predicted sentiment label (POSITIVE or NEGATIVE)
   - A confidence score indicating the model's certainty
   
Note that this medical statement is factual rather than emotionally charged, so the model's sentiment prediction may not be particularly meaningful in this context. Sentiment analysis models work best with text that expresses opinions or emotional reactions.


In [ ]:
text = "The Medical reports reveal Blood sugar and Blood pressure are high"
print(hf_model_local(text))

In [ ]:
text = "The students from this school score very high marks"
print(hf_model_local(text))

In [ ]:
text = "The Medical reports reveal Blood sugar and Blood pressure are high"
print(hf_model_cache(text))

In [ ]:
text = "The students from this school score very high marks"
print(hf_model_cache(text))


# Serialize and Save Model Information from Hugging Face Hub


This code demonstrates how to retrieve, serialize, and save detailed model information from the Hugging Face Hub:

1. **Serialization Function**: The `serialize_object()` function handles complex objects recursively:
   - Converts datetime objects to ISO format strings
   - Transforms objects with `__dict__` attributes into dictionaries
   - Processes nested lists and dictionaries
   - Preserves primitive data types

2. **API Interaction**: Creates an instance of the Hugging Face API client

3. **Model Information**: Fetches comprehensive metadata about the specified model using `api.model_info()`

4. **File Operations**: 
   - Extracts the model name from the reference path
   - Creates a JSON file named after the model
   - Serializes the model information and writes it to the file

This allows for local storage of model metadata for later reference or analysis, particularly useful for model governance, versioning, and documentation purposes.


In [ ]:
def serialize_object(obj):
    """
    Helper function to serialize custom objects like EvalResult.
    Converts objects with __dict__ attribute to dictionaries and handles datetime objects.
    """
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convert datetime to ISO 8601 string
    elif hasattr(obj, "__dict__"):
        return {key: serialize_object(value) for key, value in obj.__dict__.items()}
    elif isinstance(obj, list):
        return [serialize_object(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: serialize_object(value) for key, value in obj.items()}
    else:
        return obj  # Return the value as-is for primitive types

api = HfApi()
with open(f"models/{hf_reference.split('/')[1]}.json", "w") as json_file:
    json_file.write(json.dumps(serialize_object(api.model_info(hf_reference))))
